<a href="https://colab.research.google.com/github/sutharimanikanta/Golab/blob/main/bytePair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.4 MB/s eta 0:00:00


In [ ]:
# Using BytePair encoding from tiktoken
import importlib
import tiktoken

In [ ]:
import importlib.metadata

print("tiktoken version:", importlib.metadata.version("tiktoken"))


tiktoken version: 0.6.0


In [ ]:
tokenizer1=tiktoken.encoding_for_model("gpt-4")

In [ ]:
text="Hello, world. Is this-- a test?"

In [ ]:
integers=tokenizer1.encode(text,allowed_special={"<|endoftext|>"})
print(integers)

[9906, 11, 1917, 13, 2209, 420, 313, 264, 1296, 30]


In [ ]:
string=tokenizer1.decode(integers)
print(string)

Hello, world. Is this-- a test?


In [ ]:
tokenizer1.n_vocab

100277

Using the original Byte-pair encoding implementation used in GPT-4

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [ ]:
# !pip install bpe_openai_gpt2

In [ ]:
# import bpe_openai_gpt2
# from bpe_openai_gpt2 import get_encoder

In [ ]:
integers=tokenizer.encode(text)
tokenizer.decode(integers)

'Hello, world. Is this-- a test?'

In [ ]:
with open('/content/the-verdict.txt','r',encoding="utf-8") as f:
  raw_text=f.read()

In [ ]:
%timeit tokenizer1.encode(raw_text)

8.75 ms ± 642 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit tokenizer.encode(raw_text)

Token indices sequence length is longer than the specified maximum sequence length for this model (5145 > 2048). Running this sequence through the model will result in indexing errors


23 ms ± 660 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Embedding Layers and Linear Layers
Embedding layers in PyTorch accomplish the same as linear layers that perform matrix multiplications; the reason we use embedding layers is computational efficiency
We will take a look at this relationship step by step using code examples in PyTorch

In [ ]:
import torch

print("PyTorch version:", torch.__version__)

PyTorch version: 2.2.1+cu121


In [ ]:
idx=torch.tensor([2,3,1])
num_idx = max(idx)+1

# The desired embedding dimension is a hyperparameter
out_dim = 5

In [ ]:
torch.manual_seed(123)
embedding=torch.nn.Embedding(num_idx,out_dim)

In [ ]:
embedding.weight

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  1.5810],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015],
        [ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953]], requires_grad=True)

In [ ]:
embedding(torch.tensor([1]))

tensor([[ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

In [ ]:
onehot=torch.nn.functional.one_hot(idx)
onehot

tensor([[0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0]])

In [ ]:
torch.manual_seed(123)
linear=torch.nn.Linear(num_idx,out_dim,bias=False)

In [ ]:
print(linear)

Linear(in_features=4, out_features=5, bias=False)


Note that the linear layer in PyTorch is also initialized with small random weights; to directly compare it to the Embedding layer above, we have to use the same small random weights, which is why we reassign them here

In [ ]:
linear.weight = torch.nn.Parameter(embedding.weight.T.detach())

In [ ]:
linear(onehot.float())

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]], grad_fn=<MmBackward0>)

In [ ]:
embedding(idx)

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

* An embedding layer is typically used to map discrete categorical variables (such as words in natural language processing tasks) to continuous vector representations.
*  to perform a linear transformation on the input data, where each input neuron is connected to each output neuron by a learnable weight.
